In [ ]:
import csv
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

with open("data.csv") as file_handler:                          # Читаем файл в "list of dictionaries"
    data = list(csv.DictReader(file_handler, delimiter=','))    # tag_id:203.0,x:1.1412028310675777,y:-0.17121552147491975,
for Line in data:                                               # seq_num:10.0
    del Line['']
    del Line['time']
    for k, v in Line.items():
        Line.update({k: float(v)})         
data=sorted(data, key = lambda i: (i['tag_id'], i['seq_num']))  # Сортируем

tdata=[]                                                        # Достраиваем пропущенные точки
tLine=data[0].copy()
missed=0
for Line in data:
    if (not tdata) or (Line['tag_id']!=tdata[-1]['tag_id']):
        missed=1
    else:
        missed=Line['seq_num']-tdata[-1]['seq_num']
    if missed==1:
        tdata.append(Line)
    else:
        bLine=tdata[-1]
        for i in range(1,int(missed)):
            tLine['tag_id']=bLine['tag_id']
            tLine['x']=bLine['x']+(Line['x']-bLine['x'])*i/missed
            tLine['y']=bLine['y']+(Line['y']-bLine['y'])*i/missed
            tLine['seq_num']=bLine['seq_num']+i            
            tdata.append(tLine.copy()) 
        tdata.append(Line)
        
X=[dic['x'] for dic in tdata]
Y=[dic['y'] for dic in tdata]    

plt.figure(figsize=(13.5,7.5))
# plt.plot(X, Y,'.')
    
TagIDs=list(set([dic['tag_id'] for dic in tdata]))



ss = dict.fromkeys(['r', 'alpha', 'v', 'a', 'omega', 'epsilon', 'lf', 'af', 'xf', 'yf'], 0)   
coefs = dict.fromkeys(['cr', 'calpha', 'cv', 'ca', 'comega', 'cepsilon', 'cout'], 0)
coefs['cr']=2.5
coefs['calpha']=1.2
coefs['cv']=1.5
coefs['ca']=2.5
coefs['comega']=1.5
coefs['cepsilon']=2.5
coefs['cout']=2

for Tag in TagIDs:
    ttdata=[dic for dic in tdata if dic['tag_id'] == Tag]
    ttdataLine=ttdata.pop(0)
    X0=ttdataLine['x']
    Y0=ttdataLine['y']
    ss['xf']=X0;
    ss['yf']=Y0;    
    xout=[]
    xout.append(X0)
    yout=[]
    yout.append(Y0)
    alphaout=[]
    alphaout.append(0)
    
    for Line in ttdata:
        X=Line['x']
        Y=Line['y']
        
        ss_old=ss.copy()
        ss['r']+=(math.sqrt((X-ss['xf'])**2+(Y-ss['yf'])**2)-ss['r'])/coefs['cr']
        
        Angle=np.arctan2(Y-ss['yf'], X-ss['xf'])+2*math.pi
        if abs(Angle-ss['alpha'])>4:
            Angle-=np.sign(Angle-ss['alpha'])*2*math.pi            
        ss['alpha']+=(Angle-ss['alpha'])/coefs['calpha']
        
        ss['v']+=(ss['r']-ss_old['r']-ss['v'])/coefs['cv']
        ss['a']+=(ss['v']-ss_old['v']-ss['a'])/coefs['ca']
        ss['omega']+=(ss['alpha']-ss_old['alpha']-ss['omega'])/coefs['comega']
        ss['epsilon']+=(ss['omega']-ss_old['omega']-ss['epsilon'])/coefs['cepsilon']
        ss['lf']+=ss['v']+ss['a']/2
        ss['af']+=ss['omega']+ss['epsilon']/2
        ss['xf']+=ss['lf']*np.cos(ss['af'])/coefs['cout']
        ss['yf']+=ss['lf']*np.sin(ss['af'])/coefs['cout']
        
        xout.append(ss['xf'])
        yout.append(ss['yf'])
        alphaout.append(ss['alpha'])

#     X=[dic['x'] for dic in tdata if dic['tag_id'] == Tag]
#     Y=[dic['y'] for dic in tdata if dic['tag_id'] == Tag]

    X=[dic['x'] for dic in ttdata]
    Y=[dic['y'] for dic in ttdata]
    
    plt.figure(figsize=(13.5,7.5))
    plt.plot(X, Y,'-')
    plt.plot(xout, yout,'.')
    plt.plot(X0, Y0,'o')     
    
#     plt.plot(alphaout)  
    

    